# Example
Example grabbing and reading in [VIIRS Chlorophyll-a](https://eastcoast.coastwatch.noaa.gov/cw_viirs_chl.php) data for the Chesapeake Bay in 2024

In [31]:
import re
import requests
from bs4 import BeautifulSoup

# Define the regular expression pattern
# The pattern changed March 30, 2024
pattern = re.compile(r'(VRSUCW|VRSRCW)_2024\d{3}_DAILY_SNPP_CHLORA_CD_750M\.nc4')


def get_links(soup):    

    links = soup.find_all('a')
    matching_links = []
    for link in links:
        if pattern.search(link.get('href', '')):
            matching_links.append(link.get('href', ''))
    return matching_links

# URL of the file to download
url = 'https://www.star.nesdis.noaa.gov/pub/socd1/ecn/data/viirs/chl/daily/cd/'

#soup
response = requests.get(url)
response.raise_for_status()
html_content = response.text
soup = BeautifulSoup(html_content, 'lxml')

# Get all matching links from the page with pagination handled
matching_links = get_links(soup)

# Open the output file and write the matching links
with open('filelist.txt', 'w') as outfile:
    for link in matching_links:
        outfile.write(link + '\n')

In [32]:
import xarray as xr 
import netCDF4 as nc
import requests
import io


def process_nc4_from_stream(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors

        # Create an in-memory file-like object from the response content
        file_like = io.BytesIO(response.content)

        # Open the NetCDF4 file from the in-memory file-like object using xarray with netCDF4 engine
        with xr.open_dataset(file_like, decode_cf=False) as ds:
            # Print dataset information
            print(f'Dataset read in')
            return ds
            
    except Exception as e:
        print(f'Error processing NetCDF4 file from URL: {e}')

def read_urls_from_file(file_path, base_url):
    urls = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                # Strip 'https:' and any leading/trailing whitespace
                line = line.strip().replace('https:', '')
                # Construct the full URL by combining base URL and line content
                full_url = f"{base_url}{line}"
                urls.append(full_url)
    except Exception as e:
        print(f'Error reading URLs from file: {e}')
    return urls

# Example usage
base_url = 'https://www.star.nesdis.noaa.gov/pub/socd1/ecn/data/viirs/chl/daily/cd/'  # Replace with the actual base URL
file_path = 'filelist.txt'  # Path to the file containing the URLs

urls = read_urls_from_file(file_path, base_url)
datasets = [process_nc4_from_stream(url) for url in urls]

Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset read in
Dataset 

In [33]:
datasets[10]

<xarray.Dataset> Size: 4MB
Dimensions:      (x: 400, y: 520, time: 1, n_vals: 2, level: 1)
Coordinates:
  * x            (x) float64 3kB -8.627e+06 -8.626e+06 ... -8.295e+06 -8.294e+06
  * y            (y) float64 4kB 4.795e+06 4.794e+06 ... 4.363e+06 4.362e+06
  * time         (time) float64 8B 1.705e+09
  * level        (level) float64 8B 0.0
Dimensions without coordinates: n_vals
Data variables:
    coord_ref    int32 4B ...
    lat          (y, x) float64 2MB ...
    lon          (y, x) float64 2MB ...
    time_bounds  (time, n_vals) float64 16B ...
    chlor_a      (time, level, y, x) float32 832kB ...
Attributes:
    Conventions:           CF-1.4
    source:                SNPP_VIIRS_CHLORA
    institution:           NOAA/NESDIS Center for Satellite Applications and ...
    history:               [2024-04-13 19:58:57 UTC cwutils-3.8.0.457 2023051...
    cw:polygon_latitude:   [39.69822323 39.69822323 39.69822323 39.69822323 3...
    cw:polygon_longitude:  [-77.49422792 -76.74690596 -75.999584   -75.252262...